### Logging

In [2]:
import logging
import re
import csv
import time
from pathlib import Path
from gensim.parsing.preprocessing import * #provides a number of convenience preprocessing functions optimized for speed
from gensim.models.word2vec import Word2Vec
import gensim
from multiprocessing import cpu_count
import pandas as pd
import os
from langua import Predict
languagePredictor = Predict()
import time

from itertools import islice
#logging.getLogger('').handlers = []  #To delete previous logging configuration
from dataDictionariesLexicons import dataDict, LexiconsEnsembl


logging.basicConfig(
    format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO,
    #If log into standard output and to a file is desired:
    #handlers=[logging.FileHandler("{0}/{1}.log".format('./', uniName)), #path, File name
    #         logging.StreamHandler()]
)

from outletsBiasRatings import outletsBiasRatingsAllSides, outletAbbreviationToFullName 

outletsBiasRatings = outletsBiasRatingsAllSides
outlets = list(outletsBiasRatings.keys())
print(len(outlets),outlets)

47 ['alternet', 'democracynow', 'db', 'hp', 'theintercept', 'jacobin', 'motherjones', 'thenewyorker', 'thenation', 'slate', 'vox', 'cnn', 'nyt', 'abcnews', 'theatlantic', 'buzzfeed', 'cbs', 'economist', 'guardian', 'nbcnews', 'politico', 'timemagazine', 'wp', 'npr', 'ap', 'bbc', 'bloomberg', 'csm', 'reuters', 'thehill', 'usatoday', 'wsj', 'reason', 'we', 'wt', 'fox', 'americanspectator', 'bre', 'theblaze', 'cbn', 'dailycaller', 'dailymail', 'dailywire', 'thefederalist', 'nationalreview', 'nyp', 'newsmax']


In [22]:
# Pick 6 outlets to analyze (2 left, 2 center, 2 right) considering time range of articles
# and Allsides ratings
outlets = ['cbs','reuters','nyt', 'slate', 'csm', 'thehill', 'wt', 'nyp', 'dailymail']

## Computing years intervals

In [12]:
startYear=2000
endYear = 2019
years = list(range(startYear,endYear+1))
intervalLength = 5

intervalsList = []
interval = []

for year in range(startYear,endYear+1):
    if year%intervalLength==0:
        intervalsList.append(interval)
        interval = []
    interval.append(year)
    
intervalsList.append(interval)
    
intervalsList.pop(0)

for interval in intervalsList:
    intervalName = str(interval[0])+'-'+str(interval[-1])
    print(intervalName)
intervalsList

2000-2004
2005-2009
2010-2014
2015-2019


[[2000, 2001, 2002, 2003, 2004],
 [2005, 2006, 2007, 2008, 2009],
 [2010, 2011, 2012, 2013, 2014],
 [2015, 2016, 2017, 2018, 2019]]

In [13]:
workers = cpu_count()-2 #Number of workers to use for parallel processing. If None all available cores will be used.
print('Number of workers:', workers)

Number of workers: 14


In [23]:
current_dir = Path(os.getcwd())
base_dir = current_dir.parent
input_dir = base_dir / "scrambledArticlesForWord2vec"
model_dir = base_dir / "models"

In [24]:
import logging

def load_and_preprocess_sentences(file_path):
    try:
        with file_path.open('r', encoding='utf-8') as file:
            return [line.strip().split(' ') for line in file.readlines()]
    except Exception as e:
        logging.error(f"Failed to read from {file_path}: {e}")
        return []


def train_model(sentences, previous_model_path):
    if not sentences:
        print("No sentences provided for training.")
        return None  # Return None if no sentences to train on

    tic = time.time()
    if previous_model_path and os.path.exists(previous_model_path):
        try:
            print(f"Loading model from {previous_model_path}")
            model = Word2Vec.load(previous_model_path)
            model.build_vocab(sentences, update=True)  # Update the vocabulary
        except Exception as e:
            print(f"Error loading the model from {previous_model_path}: {e}")
            print("Training new model instead")
            model = Word2Vec(sentences, vector_size=300, window=10, min_count=5, workers=4)
            model.build_vocab(sentences)  # Building vocabulary for the new model
    else:
        print("Training new model")
        model = Word2Vec(vector_size=300, window=10, min_count=5, workers=workers, hs=0, negative = 10, sample = 0.0001, epochs=5)
        model.build_vocab(sentences)  # Building vocabulary for the new model

    model.train(sentences, total_examples=len(sentences), epochs=model.epochs)

    toc = time.time()
    computation_time = toc - tic
    print(f"Computing time for training the model: {computation_time} seconds")
    words_in_corpus = sum(len(sentence) for sentence in sentences)
    print(f"Number of words processed per second: {words_in_corpus / computation_time}")
    print(f"Most frequent words in model: {model.wv.index_to_key[:10]}")

    return model



In [25]:
for outlet in outlets:
    logging.info(outlet)
    previous_model_path = None  # Initialize the previous model path

    for interval in intervalsList:
        output_file_name = f"{interval[0]}-{interval[-1]}"
        print(f"Processing interval {output_file_name}")
        input_file_path = input_dir / outlet / f"{output_file_name}.csv"
        output_path = model_dir / outlet
        output_path.mkdir(parents=True, exist_ok=True)
        
        sentences = load_and_preprocess_sentences(input_file_path)
        if not sentences:
            print(f"No sentences found in {input_file_path}, skipping interval {output_file_name}")
            continue

        model = train_model(sentences, previous_model_path)
        if model:
            output_file_path = output_path / f"{output_file_name}.model"
            model.save(str(output_file_path))
            print(f"Model saved to {output_file_path}")
            previous_model_path = str(output_file_path)  # Update the previous model path for the next interval
        else:
            print(f"\033[1;31mSkipping training for interval {output_file_name} due to insufficient data\033[0m")
            previous_model_path = None  # Reset if training was not successful

        # Evaluation
        try:
            logging.info(" Similarity evaluation: %s", model.wv.evaluate_word_pairs('wordsim353.tsv', restrict_vocab=50000))
        except Exception as e:
            logging.error("Exception during word similarity evaluation: %s", e)
        # Analogies
        try:
            model.wv.evaluate_word_analogies('questions-words.txt', restrict_vocab=30000)
        except Exception as e:
            logging.error("Exception during word analogy evaluation: %s", e)

        logging.info("----------------------------------------------------")            
    logging.info("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX") 

2024-05-01 03:10:43,867 : INFO : cbs


Processing interval 2000-2004


2024-05-01 03:10:44,716 : INFO : Word2Vec lifecycle event {'params': 'Word2Vec<vocab=0, vector_size=300, alpha=0.025>', 'datetime': '2024-05-01T03:10:44.716234', 'gensim': '4.3.2', 'python': '3.11.5 (main, Sep 11 2023, 08:17:37) [Clang 14.0.6 ]', 'platform': 'macOS-14.4.1-arm64-arm-64bit', 'event': 'created'}
2024-05-01 03:10:44,716 : INFO : collecting all words and their counts
2024-05-01 03:10:44,716 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2024-05-01 03:10:44,909 : INFO : PROGRESS: at sentence #10000, processed 2792625 words, keeping 75162 word types


Training new model


2024-05-01 03:10:45,133 : INFO : PROGRESS: at sentence #20000, processed 5919188 words, keeping 117194 word types
2024-05-01 03:10:45,413 : INFO : PROGRESS: at sentence #30000, processed 9138063 words, keeping 151612 word types
2024-05-01 03:10:45,670 : INFO : PROGRESS: at sentence #40000, processed 12630907 words, keeping 181733 word types
2024-05-01 03:10:45,919 : INFO : PROGRESS: at sentence #50000, processed 16271370 words, keeping 208892 word types
2024-05-01 03:10:46,243 : INFO : PROGRESS: at sentence #60000, processed 19919024 words, keeping 235961 word types
2024-05-01 03:10:46,273 : INFO : collected 238302 word types from a corpus of 20224891 raw words and 60869 sentences
2024-05-01 03:10:46,274 : INFO : Creating a fresh vocabulary
2024-05-01 03:10:46,365 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=5 retains 69924 unique words (29.34% of original 238302, drops 168378)', 'datetime': '2024-05-01T03:10:46.364998', 'gensim': '4.3.2', 'python': '3.11.5 (main, Sep

...

2024-05-01 04:38:43,610 : INFO : Word2Vec lifecycle event {'fname_or_handle': '/Users/eddyji/GitRepos/01_TAD/models/dailymail/2015-2019.model', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2024-05-01T04:38:43.610633', 'gensim': '4.3.2', 'python': '3.11.5 (main, Sep 11 2023, 08:17:37) [Clang 14.0.6 ]', 'platform': 'macOS-14.4.1-arm64-arm-64bit', 'event': 'saving'}
2024-05-01 04:38:43,611 : INFO : storing np array 'vectors' to /Users/eddyji/GitRepos/01_TAD/models/dailymail/2015-2019.model.wv.vectors.npy
2024-05-01 04:38:43,676 : INFO : storing np array 'syn1neg' to /Users/eddyji/GitRepos/01_TAD/models/dailymail/2015-2019.model.syn1neg.npy
2024-05-01 04:38:43,765 : INFO : not storing attribute cum_table


Number of words processed per second: 629685.790998247
Most frequent words in model: ['year', 'said', 'he', 'view', 'comments', 'updated', 'aedt', 'company', 'money', 'new']


2024-05-01 04:38:43,903 : INFO : saved /Users/eddyji/GitRepos/01_TAD/models/dailymail/2015-2019.model
2024-05-01 04:38:43,913 : INFO : Skipping line #35 with OOV words: king	cabbage	0.23
2024-05-01 04:38:43,913 : INFO : Skipping line #37 with OOV words: king	rook	5.92
2024-05-01 04:38:43,913 : INFO : Skipping line #42 with OOV words: fuck	sex	9.44
2024-05-01 04:38:43,914 : INFO : Skipping line #55 with OOV words: movie	theater	7.92
2024-05-01 04:38:43,915 : INFO : Skipping line #87 with OOV words: food	rooster	4.42
2024-05-01 04:38:43,916 : INFO : Skipping line #97 with OOV words: rooster	voyage	0.62
2024-05-01 04:38:43,916 : INFO : Skipping line #110 with OOV words: tiger	feline	8.00
2024-05-01 04:38:43,917 : INFO : Skipping line #111 with OOV words: tiger	carnivore	7.08
2024-05-01 04:38:43,917 : INFO : Skipping line #112 with OOV words: tiger	mammal	6.85
2024-05-01 04:38:43,917 : INFO : Skipping line #114 with OOV words: tiger	organism	4.77
2024-05-01 04:38:43,917 : INFO : Skipping l

Model saved to /Users/eddyji/GitRepos/01_TAD/models/dailymail/2015-2019.model


2024-05-01 04:38:44,404 : INFO : capital-common-countries: 77.9% (296/380)
2024-05-01 04:38:44,761 : INFO : capital-world: 77.6% (294/379)
2024-05-01 04:38:44,945 : INFO : currency: 15.8% (24/152)
2024-05-01 04:38:45,326 : INFO : city-in-state: 69.8% (296/424)
2024-05-01 04:38:45,556 : INFO : family: 59.2% (181/306)
2024-05-01 04:38:46,298 : INFO : gram1-adjective-to-adverb: 27.5% (223/812)
2024-05-01 04:38:46,891 : INFO : gram2-opposite: 12.8% (90/702)
2024-05-01 04:38:48,118 : INFO : gram3-comparative: 48.3% (608/1260)
2024-05-01 04:38:48,652 : INFO : gram4-superlative: 31.8% (223/702)
2024-05-01 04:38:49,195 : INFO : gram5-present-participle: 61.5% (432/702)
2024-05-01 04:38:49,775 : INFO : gram6-nationality-adjective: 94.8% (804/848)
2024-05-01 04:38:50,912 : INFO : gram7-past-tense: 62.0% (919/1482)
2024-05-01 04:38:51,601 : INFO : gram8-plural: 72.8% (591/812)
2024-05-01 04:38:51,991 : INFO : gram9-plural-verbs: 31.6% (160/506)
2024-05-01 04:38:51,992 : INFO : Quadruplets with ou

In [26]:
import csv
from pathlib import Path

file_name_list = [f'{i[0]}-{i[-1]}' for i in intervalsList]
outlets = ['cbs','nyt', 'slate', 'reuters','thehill', 'csm', 'wt', 'nyp', 'dailymail']

def count_csv_lines(outlets, file_name_list):
    base_dir = Path("../scrambledArticlesForWord2vec")
    line_counts = {}

    csv.field_size_limit(1000000)  # 增加字段大小限制

    for outlet in outlets:
        line_counts[outlet] = {}
        outlet_dir = base_dir / outlet

        for interval in file_name_list:
            interval_dir = outlet_dir / f'{interval}.csv'
            line_count = 0

            with open(interval_dir, "r") as file:
                csv_reader = csv.reader(file)
                line_count = sum(1 for row in csv_reader)

            line_counts[outlet][interval] = line_count

    return line_counts

count = count_csv_lines(outlets, file_name_list)
count_df = pd.DataFrame(count)
count_df

,cbs,nyt,slate,reuters,thehill,csm,wt,nyp,dailymail
2000-2004,60869,421143,15313,6,6,42787,104164,146136,64491
2005-2009,155509,303699,26045,790883,37352,44535,150534,187012,170786
2010-2014,203569,460506,61621,1322954,155641,92239,513780,257433,803932
2015-2019,137473,290703,61284,1180341,220846,48144,1299226,255737,1356851
